<a id="0"></a> <br>
 # Table of Contents  
1. [Импорт](#1) 
2. [TOKEN](#2)
3. [Функции для парсинга](#3)
4. [Парсинг фильмов/сериалов/мультов/..  и базовой информации о них](#4)
5. [Парсинг рецензий](#5)
6. [Парсинг расширенной информации о фильмах](#6)

<a id="1"></a> 
## Импорт

In [1]:
# pip install kinopoisk-dev -q

In [23]:
from kinopoisk_dev import KinopoiskDev, ReviewField, ReviewParams, MovieParams, MovieField
from kinopoisk_dev.model import Movie, ReviewDocsResponseDto
import pandas as pd
import numpy as np
import time
import os

pd.set_option('display.max_columns', 99)

<a id="2"></a> 
## TOKEN заменить на свой. Сгенерировать можно здесь: https://t.me/kinopoiskdev_bot

In [2]:
#TOKEN = 

<a id="3"></a> 
## Функции для парсинга

Функция **films_base_info_parsing** перебирает в цикле заданные пользователем страницы с фильмами/сериалами/мультами или др контентом и заносит в список **content_data** по каждому фильму/сериалу/.. список с соответствующими ему данными: item.id, item.name, item.rating.kp, item.rating.imdb, item.description. Затем этот список списков преобразуется в датафрейм **content_df**, который после минимальной обработки подается на выход функции. **Тип контента** фильм/сериал/мульт/.. **можно менять** с помощью переменной type_number - 1 (movie) | 2 (tv-series) | 3 (cartoon) | 4 (anime) | 5 (animated-series) | 6 (tv-show).

In [3]:
def films_base_info_parsing(start_page, finish_page, type_number):
    kpmm = KinopoiskDev(token=TOKEN)
    content_data = []

    for i in range(start_page, finish_page):
        try:
            items = kpmm.find_many_movie(params=[
                             MovieParams(keys=MovieField.TYPE_NUMBER, value=type_number),
                             MovieParams(keys=MovieField.PAGE, value=i),
                             MovieParams(keys=MovieField.LIMIT, value='250')])    
            for item in items.docs:  # перебираем фильмы полученной страницы (250 ед фильмов/сериалов/мультов/..)
                content_data.append([item.id, item.name, item.rating.kp, item.rating.imdb, item.description])
            
        except:
            print(f'Похоже закончились запросы к апи на {i}-той странице')
            break
            
    content_df = pd.DataFrame(data=content_data, columns=['kp_id', 'name', 'kp_rating', 'imdb_rating', 'description'])  
    content_df.dropna(subset=['name'], inplace=True) # контент с такими id отсутствует/удален на кинопоиске, поэтому удаляем
    content_df = content_df.sort_values(by='kp_rating', ascending=False).reset_index(drop=True)     
            
    return content_df         

Функция **reviews_parsing** перебирает в цикле **id** фильмов/сериалов/мультов/.. из заданных пользователем строк датафрейма **content_df** и заносит в список **revs_data** по каждой рецензии список с соответствующими ей данными: item.movieId, item.id, item.userRating, item.type, item.title, item.review. Затем этот список списков преобразуется в датафрейм **revs_df**, который  подается на выход функции.

In [4]:
def reviews_parsing(content_df, start_row, finish_row, reviews_limit):
    kpr = KinopoiskDev(token=TOKEN)
    revs_data = []
    last_id = content_df.kp_id[start_row]
    
    for i in content_df.kp_id[start_row:finish_row]:
        try:
            items = kpr.review(params=[
                             ReviewParams(keys=ReviewField.MOVIE_ID, value=i),    # айди фильмов/сериалов/.. перебираются циклом
                             ReviewParams(keys=ReviewField.PAGE, value=1),               # страница, достаточно 1 
                             ReviewParams(keys=ReviewField.LIMIT, value=reviews_limit)]) # сколько рецензий берем со страницы
            for item in items.docs:  # перебираем все рецензии фильма/сериала/мульта/.. с текущим id
                revs_data.append([item.movieId, item.id, item.userRating, item.type, item.title, item.review])
            
        except:
            last_row = content_df.query('kp_id == @last_id').index
            print(f'Похоже закончились запросы к апи на {last_row[0]}-той строке. Айди последнего обработанного фильма - {last_id}')
            break
            
        last_id = i
            
    revs_df = pd.DataFrame(data=revs_data, columns=['movie_id', 'review_id', 'user_rating', 'type', 'title', 'review'])  
            
    return revs_df    

Функция **ext_info_parsing** перебирает в цикле id фильмов/сериалов/мультов/.. из заданных пользователем строк датафрейма **content_df** и заносит в список **ext_info_data** по каждому фильму список с расширенной информацией о фильме: 'kp_id', 'imdb_id', 'type', name', 'en_name', 'year', 'length', 'rating_mpaa', 'rating_kp', 'rating_imdb', 'rating_filmcritics', 'rating_rus_filmcritics', 'budget', 'fees_world', 'slogan', 'description', 'countries', 'actors', 'actors_eng', 'similar_movies', 'genres', 'production_companies', 'sequels_and_prequels'. Затем этот список списков преобразуется в датафрейм **ext_info_df**, который подается на выход функции.

In [51]:
def ext_info_parsing(content_df, start_row, finish_row):
    kp_i = KinopoiskDev(token=TOKEN)
    ext_info_data = []
    last_id = content_df.kp_id[start_row]
    
    # Проходим циклом по заданному срезу столбца content_df с id, запрашивая расширенную информацию о фильме по текущему id 
    for i in content_df.kp_id[start_row:finish_row]:
        try:
            data = kp_i.find_one_movie(i)
        
        # В случае ошибки блока try пользователю выводится уведомление об окончании кол-ва запросов к апи 
        except:
            last_row = content_df.query('kp_id == @last_id').index
            print(f'Похоже закончились запросы к апи на {last_row[0]}-той строке. Айди последнего обработанного фильма - {last_id}')
            break
            
        last_id = i

        
        # Формирование строковой переменной, которая содержит запись всех стран в одну строку, с разделением запятыми 
        countries = ''
        for i, country in enumerate(data.countries): 
            if i < len (data.countries) - 1:
                countries += str(country.name) + ', ' 
            else: 
                countries += str(country.name)
     

    
        # Формирование строковых переменных, которые содержат записи актеров/режиссеров/.. в одну строку, с разделением запятыми 
        actors = ''
        directors = ''
        producers = ''
        operators = ''
        for i, person in enumerate(data.persons):
            if person.profession == 'актеры':
                    actors += str(person.name) + ', ' 
            elif person.profession == 'режиссеры':
                    directors += str(person.name) + ', ' 
            elif person.profession == 'продюсеры':
                    producers += str(person.name) + ', ' 
            elif person.profession == 'операторы':
                    operators += str(person.name) + ', ' 
        actors = actors[:-2]
        directors = directors[:-2]
        producers = producers[:-2]
        operators = operators[:-2]

                

        # Формирование строковой переменной, которая содержит запись всех похожих фильмов и их id в одну строку, с разделением запятыми
        similar_movies = ''
        for i, movie in enumerate(data.similarMovies):
            if i < len (data.similarMovies) - 1:
                similar_movies += str(movie.id) + ' ' + str(movie.name) + ', ' 
            else: 
                similar_movies += str(movie.id) + ' ' + str(movie.name)
  


        # Формирование строковой переменной, которая содержит запись всех жанров в одну строку, с разделением запятыми
        genres = ''
        for i, genre in enumerate(data.genres): 
            if i < len (data.genres) - 1:
                genres += str(genre.name) + ', ' 
            else: 
                genres += str(genre.name)


                
        # Формирование строковой переменной, которая содержит запись всех продакшн компаний в одну строку, с разделением запятыми
        production_companies = ''
        for i, company in enumerate(data.productionCompanies): 
            if i < len (data.productionCompanies) - 1:
                production_companies += str(company.name) + ', ' 
            else: 
                production_companies += str(company.name)

                
                
        # Формирование строковой переменной, которая содержит запись всех сиквелов и приквелов в одну строку, с разделением запятыми
        sequels_and_prequels = ''
        for i, sequel in enumerate(data.sequelsAndPrequels): 
            if i < len (data.sequelsAndPrequels) - 1:
                sequels_and_prequels += str(sequel.name) + ', ' 
            else: 
                sequels_and_prequels += str(sequel.name)

                
        # Во избежание редко встречаемой ошибки со стороны апи AttributeError: 'NoneType' object has no attribute 'value' 
        # юзаю try except
        try:
            fees_world = data.fees.world.value

        except:
            fees_world = np.nan
            
    
        # На каждой итерации в список ext_info_data добавляется элемент(тоже список) с расширенными данными о фильме. 
        # Часть этих данных берется напрямую из атрибутов(data.id, data.name и пр), другая - переменные которые формировались выше(countries, actors  и пр)
        ext_info_data.append([data.id, data.externalId.imdb, data.type, data.name, data.alternativeName, data.year, 
                              data.movieLength, data.rating.kp, data.rating.imdb, data.ratingMpaa, data.rating.filmCritics, 
                              data.rating.russianFilmCritics, data.budget.value, fees_world, data.slogan, 
                              data.description, countries, actors, directors, producers, operators, similar_movies, genres, 
                              production_companies, sequels_and_prequels])
            
    ext_info_df = pd.DataFrame(data=ext_info_data, columns=['kp_id', 'imdb_id', 'type', 'name', 'en_name', 'year', 'length',
                                                            'rating_kp', 'rating_imdb', 'rating_mpaa', 'rating_filmcritics', 
                                                            'rating_rus_filmcritics', 'budget', 'fees_world', 'slogan', 
                                                            'description', 'countries', 'actors', 'directors', 'producers', 
                                                            'operators', 'similar_movies', 'genres',
                                                            'production_companies', 'sequels_and_prequels'])  
            
    return ext_info_df   

<a id="4"></a> 
## Парсинг фильмов/сериалов/мультов/..  и базовой информации о них

In [21]:
films = films_base_info_parsing(1, 5, 1)  # start_page >= 1

In [22]:
films.head(10)

,kp_id,name,kp_rating,imdb_rating,description
0,326,Побег из Шоушенка,9.108,9.3,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...
1,435,Зеленая миля,9.074,8.6,Пол Эджкомб — начальник блока смертников в тюр...
2,448,Форрест Гамп,8.919,8.8,"Сидя на автобусной остановке, Форрест Гамп — н..."
3,329,Список Шиндлера,8.843,9.0,Фильм рассказывает реальную историю загадочног...
4,535341,1+1,8.813,8.5,"Пострадав в результате несчастного случая, бог..."
5,42664,Иван Васильевич меняет профессию,8.785,8.2,Инженер-изобретатель Тимофеев сконструировал м...
6,25108,В бой идут одни «старики»,8.715,8.3,Эта эскадрилья стала «поющей» - так капитан Ти...
7,325,Крестный отец,8.709,9.2,"Криминальная сага, повествующая о нью-йоркской..."
8,42782,Операция «Ы» и другие приключения Шурика,8.705,8.5,Студент Шурик попадает в самые невероятные сит...
9,389,Леон,8.668,8.5,Профессиональный убийца Леон неожиданно для се...


In [23]:
films.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   kp_id        1000 non-null   int64  
 1   name         1000 non-null   object 
 2   kp_rating    1000 non-null   float64
 3   imdb_rating  1000 non-null   float64
 4   description  1000 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 39.2+ KB


In [29]:
films.to_csv('first_1_000_films.csv', index=False)

<a id="5"></a> 
## Парсинг рецензий

In [6]:
films_df = pd.read_csv(r'C:\ML\csv_files\first_170_films_pages_total_41_590.csv')
films_df

,kp_id,name,kp_rating,imdb_rating,description
0,1252447,Лорды раздевалки,9.399,9.3,NaN
1,1003587,Гамильтон,9.245,8.3,Рассказ о жизни американского государственного...
2,1339977,Уровень тревоги: Полночь,9.232,9.6,NaN
3,2500772,Жизнь человека. Последнее интервью,9.169,0.0,В марте 2018 года один из лучших хирургов-онко...
4,1043713,Он вам не Димон,9.127,8.8,Рассказ о предполагаемом недвижимом имуществе ...
...,...,...,...,...,...
41585,5362602,У людей так бывает,0.000,0.0,"Несколько историй о любви, в каждой из которых..."
41586,1205377,Один и тот же,0.000,4.1,"Лолу Роуз приглашают в секретное учреждение, к..."
41587,1022061,Странный дом,0.000,2.4,У пятерых друзей на пустынной дороге ломается ...
41588,1364232,Красный снег,0.000,5.1,Авторша вампирских романов приезжает на Рождес...


In [32]:
reviews = reviews_parsing(films_df, 0, 5, 250)

In [33]:
reviews

,movie_id,review_id,user_rating,type,title,review
0,326,3289333,None,Нейтральный,"Свобода, она такая.",Неразлучная парочка 'рабство-свобода' привлека...
1,326,3274753,None,Позитивный,Околозаконное рабство.,"Да-да, 'Зелёная миля', 'Побег из Шоушенка' и '..."
2,326,3266958,None,Позитивный,До сих пор хорошее кино,Долгое время этот фильм возглавлял топы лучших...
3,326,3253285,None,Позитивный,"Вольная птица порхает, где хочет!",<b>Культовая кинокартина Фрэнка Дарабонта расс...
4,326,3201243,None,Позитивный,Некоторые мысли о фильме 'Побег из Шоушенка',"Говорят, что практика делает совершенным, но е..."
...,...,...,...,...,...,...
1245,535341,1583900,None,Позитивный,None,Нужно всегда стараться находить положительные ...
1246,535341,1584073,None,Позитивный,Живое кино.,"Этот фильм появился на свет как-то тихо, без в..."
1247,535341,1584231,None,Позитивный,Эмоциональное кино для эмоциональных людей,Фильм оставил незабываемые впечатления после п...
1248,535341,1584658,None,Негативный,Фильм — диагноз,Очередной проходной фильм на актуальные социал...


In [34]:
reviews.to_csv('first_1_250_reviews.csv', index=False)

<a id="6"></a> 
## Парсинг расширенной информации о фильмах

In [4]:
films_df = pd.read_csv(r'C:\ML\csv_files\first_170_films_pages_total_41_590.csv')
films_df

,kp_id,name,kp_rating,imdb_rating,description
0,1252447,Лорды раздевалки,9.399,9.3,NaN
1,1003587,Гамильтон,9.245,8.3,Рассказ о жизни американского государственного...
2,1339977,Уровень тревоги: Полночь,9.232,9.6,NaN
3,2500772,Жизнь человека. Последнее интервью,9.169,0.0,В марте 2018 года один из лучших хирургов-онко...
4,1043713,Он вам не Димон,9.127,8.8,Рассказ о предполагаемом недвижимом имуществе ...
...,...,...,...,...,...
41585,5362602,У людей так бывает,0.000,0.0,"Несколько историй о любви, в каждой из которых..."
41586,1205377,Один и тот же,0.000,4.1,"Лолу Роуз приглашают в секретное учреждение, к..."
41587,1022061,Странный дом,0.000,2.4,У пятерых друзей на пустынной дороге ломается ...
41588,1364232,Красный снег,0.000,5.1,Авторша вампирских романов приезжает на Рождес...


In [49]:
sample = films_df.sample(10).reset_index(drop=True)

In [52]:
ext_info_df = ext_info_parsing(sample, 0, 10)
ext_info_df

Похоже закончились запросы к апи на 8-той строке. Айди последнего обработанного фильма - 676363


,kp_id,imdb_id,type,name,en_name,year,length,rating_kp,rating_imdb,rating_mpaa,rating_filmcritics,rating_rus_filmcritics,budget,fees_world,slogan,description,countries,actors,directors,producers,operators,similar_movies,genres,production_companies,sequels_and_prequels
0,1334039,None,movie,Сельский детектив. Яблоня раздора,None,2019,NaN,6.599,0.0,None,0.0,0.0,NaN,NaN,None,Олеся Федосеева – городская девушка со скучной...,Россия,"Олеся Фаттахова, Алексей Демидов, Данила Якуше...",Сахат Дурсунов,"Антон Прокопович, Татьяна Пронина, Максим Лившин",Сергей Красавцев,,"детектив, мелодрама",,"Сельский детектив 2. Месть Чернобога, Сельский..."
1,81094,tt0233325,movie,"Безумный день, или Женитьба Фигаро",None,1974,170.0,8.124,8.1,None,0.0,0.0,NaN,NaN,None,Слуга графа Альмавива весельчак и балагур Фига...,СССР,"Андрей Миронов, Александр Ширвиндт, Вера Васил...","Владимир Храмов, Валентин Плучек",В. Вершинский,Геннадий Кринитский,,комедия,,
2,229058,None,movie,Коломбо: Лебединая песня,Columbo: Swan Song,1974,94.0,7.753,8.0,None,0.0,0.0,NaN,NaN,None,Самолет популярного кантри-певца Томми Брауна ...,США,"Питер Фальк, Джонни Кэш, Айда Лупино, Джон Ден...",Николас Коласанто,"Эдвард К. Доддс, Дин Харгров, Роланд Кибби",Уильям Кронягер,,"драма, криминал, детектив, музыка",,"Коломбо: Предписание — убийство, Коломбо: Выку..."
3,579505,tt1470173,movie,Три хиджаба,Three Veils,2011,117.0,6.503,6.3,None,0.0,0.0,NaN,NaN,A drama about three young Middle Eastern women...,Действие происходит в США. В центре сюжета – и...,США,"Шитал Шетх, Анджела Зара, Мерседес Масон, Мадл...",Ролла Селбак,"Ахмад Зара, Джереми Эллиотт, Фами Фарук Фараха...",Джон Фрост,,драма,Zahra Pictures,
4,104926,tt0452623,movie,"Прощай, детка, прощай",Gone Baby Gone,2007,114.0,7.334,7.6,r,7.8,0.0,19000000.0,34612443.0,Всем нужна правда... пока они ее не находят,В пригороде Бостона без вести пропадает четыре...,США,"Кейси Аффлек, Мишель Монахэн, Морган Фриман, Э...",Бен Аффлек,"Шон Бэйли, Алан Лэдд мл., Чай Картер, Дэвид Кр...",Джон Толл,"3730 Таинственная река, 455194 Пленницы, 27988...","триллер, драма, криминал, детектив","Miramax, The Ladd Company",
5,56944,tt0045098,movie,Рим в 11 часов,Roma ore 11,1952,107.0,7.986,7.8,None,0.0,0.0,NaN,NaN,Starring Italy's most electrifying girls,В основу сюжета положен реальный факт: некий р...,"Италия, Франция","Лючия Бозе, Карла Дель Поджо, Мария Грация Фра...",Джузеппе Де Сантис,"Пол Грэц, Гоффредо Ломбардо",Отелло Мартелли,432 Похитители велосипедов,драма,"Transcontinental Films, Titanus",
6,36334,tt0054324,movie,Неоконченная песнь,Song Without End,1960,141.0,6.461,6.0,None,7.0,0.0,NaN,NaN,His Scandalous Affairs Shocked the World!,"Музыка Ференца Листа - яркая, запоминающаяся. ...",США,"Дирк Богард, Капюсин, Женевьев Паж, Патриша Мо...","Чарльз Видор, Джордж Кьюкор",Уильям Гетц,"Джеймс Вонг Хау, Чарльз Лэнг","30768 Большой вальс, 957 Интермеццо","драма, биография, музыка",William Goetz Productions,
7,2906,tt0116225,movie,Побег из Лос-Анджелеса,Escape from L.A.,1996,101.0,6.371,5.7,r,5.6,0.0,50000000.0,25477365.0,Змей вернулся,"США, 2013 год. В результате природных катаклиз...",США,"Курт Рассел, А.Дж. Ленджер, Стив Бушеми, Жорж ...",Джон Карпентер,"Дебра Хилл, Курт Рассел",Гэри Б. Кибби,"496888 Напролом, 220594 Судный день, 2277 Побе...","фантастика, боевик, триллер, приключения","Paramount, Rysher Entertainment",Побег из Нью-Йорка
8,676363,None,movie,Василиса,None,2013,100.0,5.706,4.7,None,0.0,25.0,7000000.0,NaN,Моя любовь. Моя война. Моё отечество,"Он дворянин, а она — крепостная крестьянка… Им...",Россия,"Светлана Ходченкова, Дмитрий Соломыкин, Жером ...",Антон Сиверс,"Андрей Смирнов, Юрий Сапронов, Михаил Вавилов",Илья Авербах,,"история, мелодрама",,
